In [ ]:
%matplotlib inline
import os
import numpy as np
import sys
import matplotlib
import matplotlib.pyplot as plt
c=2.998e8 #speed of light
import time
filepath='D:/measuring/'
import scipy.signal
os.chdir(filepath)

sys.path.append('H:/My Documents/measuring/')

data_dir='K:\ns\qt\Diamond\Samples\Cavity samples/2016-09 Harry_unetched\Spectrometer_Scan4NV1_ZPL'#:\measuring\data/20160503\ON diamond\pos 7/raw data spectrometer\L9'#'D:\measuring\data/20160707\CL2_newmirror_processed'#"/Users/suzannevandam/Documents/PhD/localdata/20160430/ON_diamond" 
print data_dir

%matplotlib inline

In [ ]:
import analysis.scripts.cavity.spectrometer_analysis as sa
import analysis.scripts.cavity.spectrometer_analysis_2Dsweep as spectrometer_analysis_2Dsweep

In [ ]:

sa_2D = spectrometer_analysis_2Dsweep.spectrometer_2D_analysis(data_dir,V_min=0,V_max=10,laser_wavelength=636.6e-9)
frequencies,filenumbers,intensities = sa_2D.get_data(min_frq=440,max_frq=580)#load_data_from_folder(data_dir)


In [ ]:
plt.plot(frequencies,intensities[:,0])

In [ ]:
minimum_peak_height = 700
minimum_peak_distance = 100
max_gamma = 6
x0s,u_x0s=sa_2D.peaks_from_1D_data(intensity=sa_2D.intensities[:,0],plot_peak_locations=True,
                                   max_gamma = max_gamma,
                                   minimum_peak_height=minimum_peak_height,
                                   minimum_peak_distance=minimum_peak_distance,
                                   g_gamma=0.2,
                                   plot_fit=False,
                                   fit_peaks=True)
print x0s

nu_fsrs=np.array([])

# x0s=x0s[:-1]

for l in np.arange(len(x0s)-1):
    fund_1 = x0s[l]
    fund_2 = x0s[l+1]
    nu_fsr = np.abs(fund_2 - fund_1)*1.e12
    length = c/(2*nu_fsr)
    print 'FSR is ', nu_fsr*1.e-12, 'THz'
    print 'length is ',length *1.e6 , 'um'
    nu_fsrs = np.append(nu_fsrs,nu_fsr)

print nu_fsrs
avg_nu_fsr = np.average(nu_fsrs)
u_avg_nu_fsr = np.std(nu_fsrs)
print avg_nu_fsr
avg_nu_fsrs[i,0] = avg_nu_fsr 
avg_nu_fsrs[i,1] = u_avg_nu_fsr

length = c/(2*avg_nu_fsr)

print 'length is ',length *1.e6 , 'um'

In [ ]:
nr_datafiles = 1
minimum_peak_heights = [6000,12000]
avg_nu_fsrs = np.zeros((nr_datafiles,2))
min_fsrs = np.ones(nr_datafiles)*4*1.e12
max_fsrs = np.ones(nr_datafiles)*20.e12
#max_fsrs[5] = 40.e12
#max_fsrs[6] = 40.e12
#max_fsrs[7] = 50.e12
#max_fsrs[8] = 50.e12
#min_fsrs[7] = 10.e12
#min_fsrs[8] = 10.e12

reload(sa_2D)
filename = '2016 september 07 18_01_47 OND_LT_POS2_L4 1185.csv'

sa_2D = spectrometer_analysis_2Dsweep.spectrometer_2D_analysis(data_dir,V_min=0,V_max=10,laser_wavelength=636.6e-9)
frequencies,filenumbers,intensities = sa_2D.get_data(min_frq=432,max_frq=496)#load_data_from_folder(data_dir)


for i in np.arange(nr_datafiles):
#    data_folder = os.path.join(data_dir,'L'+str(i))
    datafile = os.path.join(data_dir, filename)

    #note that what are called wavelengths, are actually the frequencies in THz! 
    wavelengths,intensities = sa.load_data(datafile)

    minimum_peak_height = minimum_peak_heights[i]
    x0s,u_x0s=sa_2D.peaks_from_1D_data(wavelengths,intensities[:,0],
                                       data_dir=datafile,save_fig=False,
                                       plot_fit =False, plot_peak_locations=True,minimum_peak_distance=10,
                                       minimum_peak_height=minimum_peak_height, g_gamma=0.1)
    print x0s

    # reinitialise every time
    nu_fsrs=np.array([])

    if i == 4:
        x0s=x0s[:-2]
    
    for l in np.arange(len(x0s)-1):
        fund_1 = x0s[l]
        fund_2 = x0s[l+1]
        nu_fsr = np.abs(fund_2 - fund_1)*1.e12
        if ((nu_fsr>min_fsrs[i]) and (nu_fsr<max_fsrs[i])):
            length = c/(2*nu_fsr)
            print nu_fsr*1.e-12, 'THz'
            print length *1.e6 , 'um'
            nu_fsrs = np.append(nu_fsrs,nu_fsr)
            
    print nu_fsrs
    avg_nu_fsr = np.average(nu_fsrs)
    u_avg_nu_fsr = np.std(nu_fsrs)
    print avg_nu_fsr
    avg_nu_fsrs[i,0] = avg_nu_fsr 
    avg_nu_fsrs[i,1] = u_avg_nu_fsr
            
        
#     fund_1 = x0s[0]
#     fund_2 = x0s[3]
#     dnu_fsr = np.abs(fund_2 - fund_1)*1.e12
#     if dnu_fsr>2*1.e12:
#         length = c/(2*dnu_fsr)
#         print dnu_fsr*1.e-12, 'THz'
#         print length *1.e6 , 'um'



# wavelengths,filenumbers,intensities = sa.load_data(data_file)


f = open(os.path.join(data_dir,'fsr_data.txt'), 'w')
np.savetxt(f,avg_nu_fsrs)
f.close()

In [ ]:
# nus = [430.99,449.51,468.4,487.4]
# nus = [431.07,449.61,468.54,487.53]
# nus = [431.3,486.64]
# nus = [438.38, 495.11]
# nus = [441.52,471.33]
# nus=[434.48,470.77]
# nus = [451.34,484.24]
nus = [438.17,467.62,497.43]
nu_fsrs = np.array([])

for i,nu in enumerate(nus):
    if i < len(nus)-1:
        print i
        nu_fsr = nus[i+1] - nus[i]
        nu_fsrs =np.append(nu_fsrs,nu_fsr)
print nu_fsrs
avg_nu_fsrs = np.average(nu_fsrs*1.e12)
u_avg_nu_fsr = np.std(nu_fsrs*1.e12)
length = c/(2*avg_nu_fsrs)
print 'cavity length is ',length


In [ ]:
636.6/2*9

In [ ]:

nu_fsrs = np.loadtxt(os.path.join(data_dir,'fsr_data.txt'))

linewidths = np.loadtxt(os.path.join(data_dir,'linewidth_data.txt'))



In [ ]:
fig,ax = plt.subplots()

ax.errorbar(nu_fsrs[:,0]*1.e-14,linewidths[:,0],xerr = nu_fsrs[:,1]*1.e-14,yerr=linewidths[:,1])
ax.set_xlabel('nu FSR (THz)')
ax.set_ylabel('linewidth (GHz)')

plt.show()

fig1,ax1 = plt.subplots()

ax1.errorbar(c/(2*nu_fsrs[:,0])*1.e6,np.divide(nu_fsrs[:,0],linewidths[:,0]*1.e9),yerr=linewidths[:,1])
ax1.set_xlabel('cavity length (um)')
ax1.set_ylabel('Finesse')


In [ ]:
reload(sa_2D)
minimum_peak_height = 6500
x0s,u_x0s=sa_2D.peaks_from_1D_data(wavelengths,intensities[:,0],data_dir=data_dir,save_fig=False,plot_fit =False, plot_peak_locations=True,minimum_peak_height=minimum_peak_height, g_gamma=1)




In [ ]:
#Finding cavity length from peaks

for i in np.arange(len(x0s)-1):
    fund_1 = x0s[i]
    fund_2 = x0s[i+1]
    dnu_fsr = np.abs(fund_2 - fund_1)*1.e12
    if dnu_fsr>3*1.e12:
        length = c/(2*dnu_fsr)
        print dnu_fsr*1.e-12, 'THz'
        print length *1.e6 , 'um'


fund_1 = x0s[0]
fund_2 = x0s[1]
dnu_fsr = np.abs(fund_2 - fund_1)*1.e12
if dnu_fsr>2*1.e12:
    length = c/(2*dnu_fsr)
    print dnu_fsr*1.e-12, 'THz'
    print length *1.e6 , 'um'


In [ ]:
###analysing SPE files